In [4]:
import getpass
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os

In [5]:
user = getpass.getuser()

nifti_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Nifti"
nifti_paths = list(Path(nifti_dir).glob('**/*.nii'))

seg_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Segmentations"
mask_r_paths = list(Path(seg_dir).glob('**/mask_R.nii.gz'))
mask_l_paths = list(Path(seg_dir).glob('**/mask_L.nii.gz'))

dst_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/Preprocessed"

In [7]:
def main():
    for nifti_path, mask_r_path, mask_l_path in zip(nifti_paths, mask_r_paths, mask_l_paths):
        nft = nib.load(nifti_path)
        nft_affine = nft.affine
        nft_imgs = nft.get_fdata()

        mask_r = nib.load(mask_r_path).get_fdata()
        mask_l = nib.load(mask_l_path).get_fdata()
        
        save_nib_file(dst_dir + '/hello.nii', nft_imgs, nft_affine)
        

def save_nib_file(dst, nft, nft_affine):
    nft_2_save = nib.Nifti1Image(nft, nft_affine)
    path = dst #make magic here pls <3
    os.makedirs(os.path.dirname(path), exist_ok=True)
    nib.save(nft_2_save, path)

#main()

In [ ]:
mask = nib.load(seg_dir + "/1001_bwt/mask_R.nii.gz").get_fdata()

In [ ]:
start_z = 0

for z in range(mask.shape[2]):
    if mask[:,:,z].any():
        start_z = z
        break

stop_z = mask.shape[2]

for z in reversed(range(mask.shape[2])):
    if mask[:,:,z].any():
        stop_z = z
        break

print("start:", start_z, "stop:", stop_z)

start_x = 0

for x in range(mask.shape[0]):
    if mask[x,:,:].any():
        start_x = x
        break

stop_x = mask.shape[0]

for x in reversed(range(mask.shape[0])):
    if mask[x,:,:].any():
        stop_x = x
        break

print("start:", start_x, "stop:", stop_x)




In [ ]:
nib_file = nib.load(nifti_paths[0])
img_affine = nib_file.affine
nib_imgs = nib_file.get_fdata()

In [ ]:
nib_imgs = nib_imgs[start_x:stop_x,50:-50, 200:201] 

low_cv, high_cv = -1024, 250

for i, im in enumerate(nib_imgs):
    if any(any(c > high_cv or c < low_cv for c in r) for r in im):
        nib_imgs[i] = [[max(min(c, high_cv), low_cv) for c in r] for r in im]

In [ ]:
for i in range(0, nib_imgs.shape[2], 17):
    img = np.rot90(nib_imgs[:, :, i])
plt.imshow(img, cmap=cm.gray)
nft_img = nib.Nifti1Image(nib_imgs, img_affine)
p = dst_dir + '/hello.nii'
os.makedirs(os.path.dirname(p), exist_ok=True)
nib.save(nft_img, p)